In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 8.0 MB/s 
     |████████████████████████████████| 287 kB 42.7 MB/s 


In [ ]:
# mount drive
from google.colab import files
from google.colab import drive

drive.mount('/content/gdrive')

import sys
sys.path.append('/content/gdrive/My Drive/scripts')

Mounted at /content/gdrive


In [ ]:
# import libraries
import CSUtilities as csu
import pandas as pd
import numpy as np
from datetime import date
import psycopg2
import time
import io

today = date.today()

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
#make a connection
con1=psycopg2.connect(
        dbname= '', 
        host='', 
        port= 0,
        user= '', 
        password= '')
cur1 = con1.cursor()

In [ ]:
# upload tasks.xlsx
uploaded_tasks = files.upload()

Saving tasks.xlsx to tasks.xlsx


In [ ]:
tasks = pd.read_excel(io.BytesIO(uploaded_tasks['tasks.xlsx']),sheet_name="data")
tasks = tasks[['groupId','groupName','FB Post Link','memberCount']]
tasks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   groupId       31 non-null     object
 1   groupName     31 non-null     object
 2   FB Post Link  31 non-null     object
 3   memberCount   31 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 1.1+ KB


In [ ]:
# drop empty kol links
df_links = tasks.dropna(subset=['FB Post Link'], how='any')
df_links.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 0 to 30
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   groupId       31 non-null     object
 1   groupName     31 non-null     object
 2   FB Post Link  31 non-null     object
 3   memberCount   31 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 1.2+ KB


In [ ]:
# upload selection.xlsx
uploaded_selection = files.upload()

Saving selection.xlsx to selection.xlsx


In [ ]:
# read selection.xlsx
df_fbids = pd.read_excel(io.BytesIO(uploaded_selection['selection.xlsx']),sheet_name="Sheet1")
df_fbids = df_fbids.loc[:,['Group ID', 'Fb Group ID']]
df_fbids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Group ID     31 non-null     object
 1   Fb Group ID  31 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 624.0+ bytes


In [ ]:
# add fb grp ids from selection xlsx
df = df_links.merge(df_fbids, left_on='groupId', right_on='Group ID', how='left')
df = df.drop(columns=['Group ID'])
df.rename(columns={'Fb Group ID':'FBGroupIDSel'}, inplace=True)
col = df.columns[:-1].insert(1, df.columns[-1])
df = df[col]
df.fillna(0,inplace=True)
df.FBGroupIDSel = df.FBGroupIDSel.astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 0 to 30
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   groupId       31 non-null     object
 1   FBGroupIDSel  31 non-null     int64 
 2   groupName     31 non-null     object
 3   FB Post Link  31 non-null     object
 4   memberCount   31 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 1.5+ KB


In [ ]:
df.head()

,groupId,FBGroupIDSel,groupName,FB Post Link,memberCount
0,586d7c85-2f43-493c-97c8-5fdc406ea1de,326430437405377,Medical Case Consultation,https://www.facebook.com/groups/medclick/perma...,636542
1,ffd2cf8f-1608-419a-b334-4042b00d3e09,1343435782401455,Home Remedies for babies and Moms 👪,https://www.facebook.com/groups/homeremediesfo...,252377
2,6df3f44f-d87a-48e8-8bcf-710aa06b56bf,334087516998050,Weight Loss Post Pregnancy,https://www.facebook.com/groups/33408751699805...,186310
3,718175d3-29fa-4e83-b60a-eca1cad397d0,364972353905531,Pregnancy Guide Group,https://m.facebook.com/groups/pregnancyfoodgui...,179984
4,4597b476-c749-470b-b813-be96355a960f,487326304778646,MoI - MOMS of INDIA™,https://www.facebook.com/groups/moi.momsofindi...,154766


In [ ]:
# groups in tasks but not in selection
set(tasks.groupId) - set(df_fbids['Group ID'])

{'3897d416-0bea-4bf9-85ee-2d90da6563a6',
 '3975401c-29bd-4a0e-ba0f-86b897973c38'}

In [ ]:
tick = time.time()

# extract post ids, create links, and format as grpid_postid
grp_post = []

for i in range(df.shape[0]):
# for i in range(2):    
    link = df['FB Post Link'][i]
    grpid, postid, LinkGrpHasId = csu.getGroupandPostIDs(link)
    if LinkGrpHasId:
      bsgrpid = grpid
      match = True if int(bsgrpid) == df.FBGroupIDSel[i] else False
    else: 
      bsgrpid = df['FBGroupIDSel'][i]
      match = True
    fbgrpidSel_str = str(df.FBGroupIDSel[i])
    grp_post.append([df.groupId[i], fbgrpidSel_str, df.groupName[i], grpid, postid, LinkGrpHasId, grpid,\
                     f'https://www.facebook.com/groups/{bsgrpid}/permalink/{postid}/', f'{bsgrpid}_{postid}', match])
print(f'time elapsed: {time.time() - tick} seconds')

time elapsed: 0.004792451858520508 seconds


In [ ]:
# create dataframe
postid_df = pd.DataFrame(grp_post, columns=['CSGroupID','FbGrpIdSel','GroupName','LinkGrpID','PostID',\
                                            'LinkHasGrpID','GrpIDfromLink','FinalLink','grp_post', 'FbIdsMatch'])
postid_df.sort_values('FbIdsMatch',ascending=False,inplace=True)

In [ ]:
# wrong kol links
postid_df[(postid_df.FbIdsMatch == False) | (postid_df.FbGrpIdSel == '0')]

,CSGroupID,FbGrpIdSel,GroupName,LinkGrpID,PostID,LinkHasGrpID,GrpIDfromLink,FinalLink,grp_post,FbIdsMatch
24,3897d416-0bea-4bf9-85ee-2d90da6563a6,0,Mental Health Professionals,meetcounsellor.org,3190880017822089,False,meetcounsellor.org,https://www.facebook.com/groups/0/permalink/31...,0_3190880017822089,True
30,3975401c-29bd-4a0e-ba0f-86b897973c38,0,Nutrition and Dietetics,113018978774379,5139865842756309,True,113018978774379,https://www.facebook.com/groups/11301897877437...,113018978774379_5139865842756309,False


In [ ]:
kol = postid_df.grp_post
kol_tuple = tuple(kol)
print(len(kol_tuple))
kol_tuple

31


('326430437405377_5127490533965986',
 '414473255965163_1217585942320553',
 '792991647424136_5195887950467795',
 '125992824632211_1108467646384719',
 '238534840545628_683171569415284',
 '2238153679742618_3937697866454849',
 '2141010606113725_3381694438711996',
 '0_3190880017822089',
 '561760821218711_1105345760193545',
 '135599973772163_1011714799494005',
 '1078984939205613_1446773219093448',
 '443092582754459_1513793329017707',
 '555119465169437_1005901676757878',
 '141888199860372_980074459375071',
 '1970000843317878_3172331689751448',
 '486291502059374_993339494687903',
 '1343435782401455_5091453930932936',
 '431073793970737_1382324108845696',
 '2751637595114137_3195983067346252',
 '240400049476826_2018621208321359',
 '232925793579287_1807162146155636',
 '345660702440879_1700844310255838',
 '131392557542254_970122727002562',
 '268934937907540_569707517830279',
 '629828977483310_1383187965480737',
 '124575661602415_1111187509607887',
 '448126775237983_5296566120394000',
 '487326304778

In [ ]:
date_range = csu.dateRange('2022-06-02', '2022-06-24')
date_range

"createdatutc >= '2022-06-02' and createdatutc <= '2022-06-24 23:59:59'"

In [ ]:
a1= f"""
(
select groupid, textlower,fbpermlink,parentsourceid as commentid,sourceid as postid,type,
count,reactiontype, sentimentrvalue, recordedatutc,createdatutc from bd_cs_prod_conversations  
where parentsourceid in {kol_tuple}
 and (
        {date_range}
      )

and (grouptype = 'Facebook')
)
union
(
select groupid,textlower,fbpermlink,parentsourceid as commentid,sourceid as postid,type,
count,reactiontype, sentimentrvalue, recordedatutc,createdatutc from bd_cs_prod_conversations  
where sourceid in {kol_tuple}
 and (
        {date_range}
      )

and (type = 'Post' or type = 'Reaction')
and (grouptype = 'Facebook')
)
"""

In [ ]:
%%time
cur1.execute(a1)
w = cur1.fetchall()
y = pd.DataFrame(w)
y.columns = ['groupid','textlower','fbpermlink','commentid','postid','type','count','reactiontype','sentimentrvalue', 'recordedatutc','createdatutc']

kol_y = y.copy()

CPU times: user 201 ms, sys: 35.5 ms, total: 236 ms
Wall time: 28.3 s


In [ ]:
# y = kol_y.copy()

In [ ]:
y.shape

(5652, 11)

In [ ]:
y.to_excel(f'koldump {today}.xlsx', index=False)
files.download(f'koldump {today}.xlsx')

In [ ]:
# posts count
posts = y[y.type == 'Post']

#comments count
kol_comments = y[y.type == 'Comment']

# hashtags comments count

y14 = kol_comments[kol_comments["textlower"].str.contains("#")]

##reactions count
y12 = y[y.type == 'Reaction']
y1 = y12[['fbpermlink','count','recordedatutc','reactiontype','groupid']]
lol = y1.loc[y1.groupby(['groupid','fbpermlink','reactiontype']).recordedatutc.idxmax()]
lol1=lol[['groupid','count']]
kol_reactions = lol1.astype(dtype={'count':'int64'})
qq=kol_reactions[['count']].sum()

# print('KOL Posts:', posts.shape[0])
# print('KOL Comments:', kol_comments.shape[0])
# print('KOL Comments #:',y14.shape[0])
# print('KOL Reactions:',qq[0])

kol_output = pd.DataFrame(np.array([['Posts', posts.shape[0]], ['Comments', \
                                   kol_comments.shape[0]], ['Comments #',y14.shape[0]], ['Reactions',qq[0]]]), \
                         columns=['content','kol'])
kol_output.head()

,content,kol
0,Posts,18
1,Comments,1285
2,Comments #,711
3,Reactions,989


In [ ]:
# kol_posts_groupwise = posts.groupby('groupid')['postid'].count()
# kol_posts_groupwise

In [ ]:
# hashtag kol comments
# kol_comments = y14.copy()
# kol_comments.shape

(792, 11)

In [ ]:
kol_comments_groupwise = kol_comments.groupby(['groupid'])[['groupid']].count()
kol_comments_groupwise.rename(columns={'groupid':'kol_comments'},inplace=True)
kol_comments_groupwise.reset_index(inplace=True)

kol_react_groupwise = kol_reactions.groupby(['groupid'])[['count']].sum()
kol_react_groupwise = kol_react_groupwise.rename(columns={'count':'kol_reactions'}).reset_index()

kol_comments_reactions_grpwise = kol_comments_groupwise.merge(kol_react_groupwise, on='groupid', how='outer').fillna(0)
kol_comments_reactions_grpwise

,groupid,kol_comments,kol_reactions
0,0b5d6a5d-4636-49c9-ae4e-7c3f139d7bcc,16,11.0
1,135e9c9a-4f61-4748-8584-48184e731d15,28,7.0
2,1ba40bb4-7329-453d-a535-20c256a24278,136,0.0
3,1c304253-bb8e-440c-ae5e-5bb827d837f3,13,0.0
4,2c806dc3-432f-489f-a00e-dbaa270a8de6,10,0.0
5,3897d416-0bea-4bf9-85ee-2d90da6563a6,23,64.0
6,3975401c-29bd-4a0e-ba0f-86b897973c38,37,26.0
7,39825aeb-1092-4275-9e65-b38e8c60609a,60,0.0
8,3d51ebbe-37d1-47a6-90e1-654d60d737ed,62,0.0
9,4597b476-c749-470b-b813-be96355a960f,9,53.0


In [ ]:
kol_comments_reactions_grpwise.to_excel('grpwise_kol_react_comments.xlsx', index=False)
files.download('grpwise_kol_react_comments.xlsx')

In [ ]:
# y[y.textlower.str.contains('#clothesasgoodasnew')].shape[0]

In [ ]:
#reactions count dump
lol1.to_excel("reactionsaditya.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
##shares count
y12 = y[y.type == 'PostsSharedCount']
y1 = y12[['fbpermlink','recordedatutc','reactiontype','groupid']]
y1['recordedatutc'] = pd.to_datetime(y1['recordedatutc'])
lol = y1.loc[y1.groupby(['fbpermlink']).recordedatutc.idxmax()]
lol1=lol[['fbpermlink','Count']]
lol1["Count"] = pd.to_numeric(lol1["Count"])
qq=lol1[['Count']].sum()
qq

In [ ]:
#shares dump
lol1.to_excel("share.xlsx")
files.download('share.xlsx')

## Combined kol & ugc

In [ ]:
# generic

# group_ids = tuple(pd.read_excel('generic.xlsx')['groupid'])
# len(group_ids)

In [ ]:
# Group IDs

group_ids = tuple(tasks['groupId'])
print('Groups count:',len(group_ids))
# print(str(group_ids[0][:2]) + str(group_ids[-1][-2:]))

Groups count: 31


In [ ]:
hashtags = csu.makeTransformations('#Tata1mg, #1mg,#1mgforevery1,#1mgforeveryone,#BringingCareToHealth,#RightDiagnosticForAll,#RightDiagnosticsForAll', model=False)
hashtags

"TEXTLOWER ILIKE '%#tata1mg%' OR TEXTLOWER ILIKE '%#1mg%' OR TEXTLOWER ILIKE '%#1mgforevery1%' OR TEXTLOWER ILIKE '%#1mgforeveryone%' OR TEXTLOWER ILIKE '%#bringingcaretohealth%' OR TEXTLOWER ILIKE '%#rightdiagnosticforall%' OR TEXTLOWER ILIKE '%#rightdiagnosticsforall%'"

In [ ]:
# Combined Admin & UGC- reactions & comments

a1=f"""
(
  select
  groupid,
  fbpermlink,
  rawtext,
  parentsourceid as commentid,
  sourceid as postid,
  type,
  count,
  updatedatutc,
  createdatutc,
  recordedatutc,
  reactiontype
  from
  bd_cs_prod_conversations
  where
  fbpermlink in (
    Select
      fbpermlink
    from
      bd_cs_prod_conversations
    where
      ({hashtags})
      and groupid in {group_ids}
      and (
        {date_range}
      )
      and (type = 'Post')
      and (grouptype = 'Facebook')
  )
)
"""

In [ ]:
%%time
cur1.execute(a1)
w = cur1.fetchall()
y = pd.DataFrame(w)
y.columns = ['groupid','fbpermlink','rawtext','commentid','postid','type','Count','updatedatutc','createdatutc','recordedatutc','reactiontype']

combined_y = y.copy()

CPU times: total: 234 ms
Wall time: 4min 6s


In [ ]:
y.shape

(6178, 11)

In [ ]:
y[y.type=='Post']['rawtext'].head()

13    Collaboration #Tata1mg #1mg #1mgforevery1 #Bri...
42    Collaboration #Tata1mg #1mg #1mgforevery1 #Bri...
44    Collaboration #Tata1mg #1mg #1mgforevery1 #Bri...
48    Collaboration #Tata1mg #1mg #1mgforevery1 #Bri...
49    Collaboration #Tata1mg #1mg #1mgforevery1 #Bri...
Name: rawtext, dtype: object

In [ ]:
#kol+ugc posts with hashtags
y1 = y[y.type == 'Post']
y2=y1[['groupid','postid','rawtext','fbpermlink']]
y2.shape[0]

326

## UGC - User Generated Content

In [ ]:
combined = y2.postid.drop_duplicates().to_frame()
combined.head()

,postid
13,414473255965163_1217585942320553
42,141888199860372_980074459375071
44,334087516998050_1277076532699139
48,232925793579287_1807162146155636
49,131392557542254_970122727002562


In [ ]:
ugc = combined[~combined.isin(kol_tuple)].dropna()
kols_filtered = combined[combined.isin(kol_tuple)].dropna()
print('UGC posts:',ugc.shape[0])
print('KOL posts filtered:',kols_filtered.shape[0])
print('Combined:',combined.shape[0])

UGC posts: 282
KOL posts filtered: 28
Combined: 310


In [ ]:
# kol posts not in combined
# kol[~kol.isin(kols_filtered.postid)].reset_index(drop=True)

In [ ]:
ugc_tuple = tuple(ugc.postid)
print(len(ugc))
ugc_tuple[:5]

282


('268934937907540_571390730995291',
 '141888199860372_980173759365141',
 '141888199860372_980171126032071',
 '141888199860372_980193576029826',
 '345660702440879_1701022106904725')

In [ ]:
a1= f"""
(
select groupid, textlower,fbpermlink,parentsourceid as commentid,sourceid as postid,type,
count,reactiontype, sentimentrvalue, recordedatutc,createdatutc from bd_cs_prod_conversations  
where parentsourceid in {ugc_tuple}
 and (
        {date_range}
      )

and (grouptype = 'Facebook')
)
union
(
select groupid,textlower,fbpermlink,parentsourceid as commentid,sourceid as postid,type,
count,reactiontype, sentimentrvalue, recordedatutc,createdatutc from bd_cs_prod_conversations  
where sourceid in {ugc_tuple}
 and (
        {date_range}
      )

and (type = 'Post' or type = 'Reaction')
and (grouptype = 'Facebook')
)
"""

In [ ]:
%%time
cur1.execute(a1)
w = cur1.fetchall()
y = pd.DataFrame(w)
y.columns = ['groupid','textlower','fbpermlink','commentid','postid','type','count','reactiontype','sentimentrvalue', 'recordedatutc','createdatutc']
ugc_y = y.copy()

CPU times: total: 46.9 ms
Wall time: 14.7 s


In [ ]:
# y = ugc_y.copy()

In [ ]:
y.shape

(5652, 11)

In [ ]:
csu.printReferenceConv(y[~(y.type=='Reaction')], 0, 5)

Reference Conversations
────────────────────────────────────────────────────────────────────────────────
Preeti Khurana Chawla thanx
────────────────────────────────────────────────────────────────────────────────
Meena S Goyal thanx for sharing
────────────────────────────────────────────────────────────────────────────────
#Tata1mg #1mg #1mgforevery1 #BringingCareToHealth #RightDiagnosticForAll
Tata 1 mg is a very good healthcare..I must try it
────────────────────────────────────────────────────────────────────────────────
#Tata1mg #1mg #1mgforevery1 #BringingCareToHealth #RightDiagnosticForAll

Amazing app and service
────────────────────────────────────────────────────────────────────────────────
#Tata1mg #1mg #1mgforevery1 #BringingCareToHealth #RightDiagnosticForAll
Best one yet. Thanks for sharing mam amezing app highly recommended
────────────────────────────────────────────────────────────────────────────────


In [ ]:
y.to_excel(f'ugcdump {today}.xlsx', index=False)
files.download(f'ugcdump {today}.xlsx')

In [ ]:
# posts count
posts = y[y.type == 'Post'].drop_duplicates(subset=['postid'])

#comments count
ugc_comments = y[y.type == 'Comment']

# hashtags comments count

y14 = ugc_comments[ugc_comments["textlower"].str.contains("#")]

##reactions count
y12 = y[y.type == 'Reaction']
y1 = y12[['fbpermlink','count','recordedatutc','reactiontype','groupid']]
lol = y1.loc[y1.groupby(['groupid','fbpermlink','reactiontype']).recordedatutc.idxmax()]
lol1=lol[['groupid','count']]
ugc_reactions = lol1.astype(dtype={'count':'int64'})
qq=ugc_reactions[['count']].sum()

# print('UGC Posts:', posts.shape[0])
# print('UGC Comments:', ugc_comments.shape[0])
# print('UGC Comments #:',y14.shape[0])
# print('UGC Reactions:',qq[0])

ugc_output = pd.DataFrame(np.array([['Posts', posts.shape[0]], ['Comments', \
                                   ugc_comments.shape[0]], ['Comments #',y14.shape[0]], ['Reactions',qq[0]]]), \
                         columns=['content','ugc'])
ugc_output.head()

,content,ugc
0,Posts,18
1,Comments,1285
2,Comments #,711
3,Reactions,989


In [ ]:
ugc_posts_groupwise = posts.groupby(['groupid'])[['groupid']].count()
ugc_posts_groupwise.rename(columns={'groupid':'ugc_posts'},inplace=True)
ugc_posts_groupwise.reset_index(inplace=True)
ugc_posts_groupwise

,groupid,ugc_posts
0,0b5d6a5d-4636-49c9-ae4e-7c3f139d7bcc,1
1,3897d416-0bea-4bf9-85ee-2d90da6563a6,1
2,3975401c-29bd-4a0e-ba0f-86b897973c38,1
3,4597b476-c749-470b-b813-be96355a960f,1
4,485a579e-816c-47c5-bdad-0436f5f58d31,1
5,51414c4c-168a-4b61-9c54-0830ebe470ec,1
6,589c9caf-f9ea-456f-a782-d8214e1a54c3,1
7,5e2df1d9-a783-4fe8-86e7-5584a1425acd,1
8,64771ffc-48af-430b-86c1-863022d52d66,1
9,65ebfc20-0ba7-433d-b14b-74163e4f27dd,1


In [ ]:
ugc_posts_groupwise.to_excel('grpwise_ugc_posts.xlsx')
files.download('grpwise_ugc_posts.xlsx')

In [ ]:
# ugc_comments = y14.copy()
# ugc_comments.shape

In [ ]:
ugc_comments_groupwise = ugc_comments.groupby(['groupid'])[['groupid']].count()
ugc_comments_groupwise.rename(columns={'groupid':'ugc_comments'},inplace=True)
ugc_comments_groupwise.reset_index(inplace=True)

ugc_react_groupwise = ugc_reactions.groupby(['groupid'])[['count']].sum()
ugc_react_groupwise = ugc_react_groupwise.rename(columns={'count':'ugc_reactions'}).reset_index()

ugc_comments_reactions_grpwise = ugc_comments_groupwise.merge(ugc_react_groupwise, on='groupid', how='outer').fillna(0)
ugc_comments_reactions_grpwise

,groupid,ugc_comments,ugc_reactions
0,0b5d6a5d-4636-49c9-ae4e-7c3f139d7bcc,16,11.0
1,135e9c9a-4f61-4748-8584-48184e731d15,28,7.0
2,1ba40bb4-7329-453d-a535-20c256a24278,136,0.0
3,1c304253-bb8e-440c-ae5e-5bb827d837f3,13,0.0
4,2c806dc3-432f-489f-a00e-dbaa270a8de6,10,0.0
5,3897d416-0bea-4bf9-85ee-2d90da6563a6,23,64.0
6,3975401c-29bd-4a0e-ba0f-86b897973c38,37,26.0
7,39825aeb-1092-4275-9e65-b38e8c60609a,60,0.0
8,3d51ebbe-37d1-47a6-90e1-654d60d737ed,62,0.0
9,4597b476-c749-470b-b813-be96355a960f,9,53.0


In [ ]:
ugc_comments_reactions_grpwise.to_excel('grpwise_ugc_react_comments.xlsx')
files.download('grpwise_ugc_react_comments.xlsx')

## Combined

In [ ]:
# combined kol and ugc
total = kol_output.merge(ugc_output, on='content')
total = total.astype({'kol':'int64','ugc':'int64'})
total['combined'] = total['kol'] + total['ugc']
total

,content,kol,ugc,combined
0,Posts,18,18,36
1,Comments,1285,1285,2570
2,Comments #,711,711,1422
3,Reactions,989,989,1978


In [ ]:
combined_comments_reactions_grpwise = kol_comments_reactions_grpwise.merge(ugc_comments_reactions_grpwise,on='groupid',how='outer').fillna(0)
combined_comments_reactions_grpwise['comments'] = combined_comments_reactions_grpwise.kol_comments + combined_comments_reactions_grpwise.ugc_comments
combined_comments_reactions_grpwise['reactions'] = combined_comments_reactions_grpwise.kol_reactions + combined_comments_reactions_grpwise.ugc_reactions
combined_comments_reactions_grpwise

,groupid,kol_comments,kol_reactions,ugc_comments,ugc_reactions,comments,reactions
0,0b5d6a5d-4636-49c9-ae4e-7c3f139d7bcc,16,11.0,16,11.0,32,22.0
1,135e9c9a-4f61-4748-8584-48184e731d15,28,7.0,28,7.0,56,14.0
2,1ba40bb4-7329-453d-a535-20c256a24278,136,0.0,136,0.0,272,0.0
3,1c304253-bb8e-440c-ae5e-5bb827d837f3,13,0.0,13,0.0,26,0.0
4,2c806dc3-432f-489f-a00e-dbaa270a8de6,10,0.0,10,0.0,20,0.0
5,3897d416-0bea-4bf9-85ee-2d90da6563a6,23,64.0,23,64.0,46,128.0
6,3975401c-29bd-4a0e-ba0f-86b897973c38,37,26.0,37,26.0,74,52.0
7,39825aeb-1092-4275-9e65-b38e8c60609a,60,0.0,60,0.0,120,0.0
8,3d51ebbe-37d1-47a6-90e1-654d60d737ed,62,0.0,62,0.0,124,0.0
9,4597b476-c749-470b-b813-be96355a960f,9,53.0,9,53.0,18,106.0


In [ ]:
combined_comments_reactions_grpwise.to_excel('grpwise_engagement.xlsx')
files.download('grpwise_engagement.xlsx')

In [ ]:
combined_comments_reactions_grpwise.loc[:,['groupid','comments','reactions']].to_clipboard(index=False)
combined_comments_reactions_grpwise.loc[:,['groupid','comments','reactions']]

,groupid,comments,reactions
0,046b62aa-72ec-48e5-be7e-8625ba91d5dc,497.0,713.0
1,11992a90-d23e-41ff-b16b-7b1304eb9f7b,32.0,99.0
2,19e4aa07-8395-40a6-b133-2f380fe3258b,788.0,920.0
3,1e006a60-14f6-43c0-8421-4b367f19a4aa,32.0,55.0
4,219c8f04-ba5a-4e49-a2a1-a9cab934e059,379.0,681.0
5,22fe26d5-3237-4200-a1e9-c84fddc06105,568.0,1137.0
6,2820246a-6c1f-4628-b840-03971cee7766,526.0,778.0
7,2d9e9b5f-daa0-4c33-b14a-4cd22a32992f,137.0,262.0
8,2da980be-d2b7-44de-ae8f-72fd1f8d842b,540.0,1086.0
9,36da0082-f835-4814-82b4-8d6abd9b6fde,12.0,35.0
